<a href="https://colab.research.google.com/github/saisrikanthmadugula/rkAMM-Simulation-Framework/blob/main/Run_Simulation_on_Esteva_(2023)_Data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import sys

# --- AGENT DEFINITIONS (From our framework) ---

class PricingAgent:
    """
    Our framework's Pricing Agent.
    Uses the Reverse Kelly AMM logic from Esteva et al. (2023)
    to calculate the required premium for a loan.
    """
    def __init__(self):
        pass

    def calculate_premium(self, p_d, collateral_level, loan_amount=1.0):
        """
        Calculates the premium (P) based on rkAMM logic.
        """
        L = loan_amount * (1.0 - collateral_level)

        if L <= 0 or p_d <= 0:
            return 0.0, L
        if p_d >= 1.0:
            return np.inf, L

        # The core rkAMM formula (Esteva et al., 2023)
        premium = (L * p_d) / (1.0 - p_d)

        return premium, L

class BlockchainEnforcementAgent:
    """
    Our framework's Enforcement Agent.
    Simulates the execution of the loan on the blockchain.
    """
    def __init__(self):
        pass

    def execute_loan(self, p_d):
        """
        Simulates if a loan defaults or is repaid based on its p_d.

        Returns:
        str: "Repaid" or "Default"
        """
        return "Repaid" if np.random.rand() >= p_d else "Default"

# --- SIMULATION ENGINE ---

def run_simulation_on_data(df_invoices, collateral_level):
    """
    Applies our Pricing and Enforcement Agents to the loaded Esteva data.
    """
    print(f"  Running simulation with C = {collateral_level*100:.0f}%...")

    pricing_agent = PricingAgent()
    blockchain_agent = BlockchainEnforcementAgent()

    results = []

    total_loans = len(df_invoices)

    for i, row in df_invoices.iterrows():
        # 1. Get p_d from the Esteva dataset
        p_d = row['p_d']

        # 2. Our Pricing Agent calculates the premium
        premium, loss_given_default = pricing_agent.calculate_premium(
            p_d, collateral_level
        )

        # 3. Our Enforcement Agent simulates the outcome
        outcome = blockchain_agent.execute_loan(p_d)

        # 4. Calculate profit/loss
        if outcome == "Repaid":
            net_profit_loss = premium
        else: # Default
            net_profit_loss = premium - loss_given_default

        results.append({
            "loan_id": i,
            "p_d": p_d,
            "collateral_level": collateral_level,
            "premium": premium,
            "loss_given_default": loss_given_default,
            "outcome": outcome,
            "net_profit_loss": net_profit_loss
        })

        # Print progress
        if (i + 1) % 1000 == 0:
            sys.stdout.write(f"\r    ...processed {i+1}/{total_loans} loans.")
            sys.stdout.flush()

    print("\n  ...Run complete.")
    return pd.DataFrame(results)

# --- SCRIPT EXECUTION ---

if __name__ == "__main__":

    # This is the dataset used in the Esteva et al. (2023) notebooks
    DATA_URL = "https://raw.githubusercontent.com/ballesterosbr/rkAMM/master/notebooks/simulated/l_invoices_10k_0.6_0.3_0.01.csv"

    print(f"Downloading Esteva et al. (2023) dataset from:\n{DATA_URL}\n")

    try:
        df_invoices = pd.read_csv(DATA_URL)
        print(f"Successfully loaded {len(df_invoices)} invoices.")
        print("Dataset 'p_d' distribution (as per Esteva et al.):")
        print(df_invoices['p_d'].value_counts(normalize=True))
        print("\n" + "="*40 + "\n")

    except Exception as e:
        print(f"Error: Could not download or read the file. {e}")
        print("Please check the URL or your internet connection.")
        sys.exit(1)

    # --- Run Scenarios ---
    # We will test our framework on their data using the two
    # collateral levels from your V-11 paper.
    scenarios_to_run = [0.0, 0.6]
    scenario_results = {}

    for c_level in scenarios_to_run:
        df_result = run_simulation_on_data(df_invoices, c_level)
        scenario_results[c_level] = df_result

        print(f"\n--- Financial Summary (C = {c_level*100:.0f}%) ---")
        total_premiums = df_result['premium'].sum()
        total_defaults = len(df_result[df_result['outcome'] == 'Default'])
        total_losses_paid = df_result[df_result['outcome'] == 'Default']['loss_given_default'].sum()
        net_profit = df_result['net_profit_loss'].sum()

        print(f"  Total Premiums Collected: {total_premiums:,.2f}")
        print(f"  Total Defaults:         {total_defaults} / {len(df_result)}")
        print(f"  Total Losses Paid:      {total_losses_paid:,.2f}")
        print(f"  NET PROFIT/LOSS:        {net_profit:,.2f}")

        if net_profit > 0:
            print("  Result: The LP pool was profitable.")
        else:
            print("  Result: The LP pool was unprofitable.")
        print("\n" + "="*40 + "\n")

    print("All simulations on Esteva et al. (2023) data are complete.")